In [3]:
#Get data for streets, driving, biking, walk paths
import osmnx as ox

#Library for analyziing complex networks and graphs
import networkx as nx

#Converts address to latitude and longitude
from geopy.geocoders import Nominatim

#Interactive Graphs
import folium

#Static Graphing
import matplotlib.pyplot as plt

#Display final map
from IPython.display import display

#Memory Efficent tools for working with iterations
import itertools

#Methods to get Geopy.geocoders to work
import ssl
import certifi

In [7]:
# Create an SSL context that uses certifi's certificate bundle
ssl_context = ssl.create_default_context(cafile=certifi.where())

# Initialize Geolocator for the coordinates
geolocator = Nominatim(user_agent='wichita_nav', ssl_context=ssl_context)

# Generate road network for Wichita
G = ox.graph_from_place('Wichita, Kansas, USA', network_type='drive')

# Function to plot paths on the map
def plot_route(G, route, route_map, color='blue'):
    route_points = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
    folium.PolyLine(route_points, color=color, weight=4.5, opacity=0.7).add_to(route_map)

# Function to get nearest node from a location
def get_nearest_node(address): 
    location = geolocator.geocode(address, timeout=5)  # 5 sec timeout 
    if location:
        print(f"Geocoded Address: {address} -> ({location.latitude}, {location.longitude})")  # Debugging output
        return ox.distance.nearest_nodes(G, X=location.longitude, Y=location.latitude)
    else: 
        print(f"Failed to geocode address: {address}")  # Identify the issue
        return None

# Example start and end addresses
start_address = "7331 Ayesbury Cir, Wichita KS"
end_address = "123 Douglas Ave, Wichita, KS"

start_node = get_nearest_node(start_address)
end_node = get_nearest_node(end_address)

if start_node and end_node:
    # Interactive map
    route_map = folium.Map(location=[37.6872, -97.3301], zoom_start=12)  # Centering on Wichita

    # Route 1 shortest path using Dijkstra's algorithm
    shortest_route = nx.shortest_path(G, start_node, end_node, weight='length')
    plot_route(G, shortest_route, route_map, color='blue')

    # Create copy of original data
    G_penalized = G.copy()

    # Adjust weights to favor main roads
    for u, v, data in G_penalized.edges(data=True):
        road_type = data.get('road_type', 'local')  # Default to local if missing
        if road_type in ['highway', 'arterial']:
            data['length'] *= 0.5  # Reduce weight more for main roads
        elif road_type in ['residential', 'local']:
            data['length'] *= 2.5  # Further increase weight for smaller roads

    # Penalize the first route to discourage reuse
    for u, v in zip(shortest_route[:-1], shortest_route[1:]):
        if G_penalized.has_edge(u, v):
            for key, data in G_penalized[u][v].items():
                original_length = data.get('length', 1)
                road_type = data.get('road_type', 'local')
                
                # Apply stronger penalty for minor roads, moderate for major roads
                if road_type in ['residential', 'local']:
                    penalty = 4.5
                elif road_type in ['highway', 'arterial']:
                    penalty = 1.4
                else:
                    penalty = 1.5
                
                G_penalized[u][v][key]['length'] = original_length * penalty
                print(f"Penalized edge ({u}, {v}, key={key}): {original_length} -> {G_penalized[u][v][key]['length']}")

    # Compute the second route on the penalized graph
    second_route = nx.shortest_path(G_penalized, start_node, end_node, weight='length')
    plot_route(G, second_route, route_map, color='red')
    
    folium.Marker([G.nodes[start_node]['y'], G.nodes[start_node]['x']], popup='Start').add_to(route_map)
    folium.Marker([G.nodes[end_node]['y'], G.nodes[end_node]['x']], popup='End').add_to(route_map)

    # Display map
    display(route_map)
else:
    print("Error: One of the addresses could not be geocoded.")


Geocoded Address: 7331 Ayesbury Cir, Wichita KS -> (37.727137306122444, -97.25127428571429)
Geocoded Address: 123 Douglas Ave, Wichita, KS -> (37.68591687755102, -97.33855526530613)
Penalized edge (122229824, 122284236, key=0): 257.1500614891194 -> 1157.1752767010373
Penalized edge (122284236, 122284222, key=0): 238.98777388119825 -> 1075.444982465392
Penalized edge (122284222, 122344313, key=0): 116.8239884735985 -> 525.7079481311932
Penalized edge (122344313, 122459139, key=0): 209.57829174346392 -> 943.1023128455877
Penalized edge (122459139, 674282359, key=0): 44.28352943200812 -> 199.27588244403654
Penalized edge (122459139, 674282359, key=1): 123.31485124723773 -> 554.9168306125698
Penalized edge (122459139, 674282359, key=2): 136.61045970835664 -> 614.747068687605
Penalized edge (674282359, 122438959, key=0): 632.3555018725694 -> 2845.5997584265624
Penalized edge (122438959, 122221221, key=0): 604.7444863441622 -> 2721.35018854873
Penalized edge (122221221, 122459110, key=0): 30